# Analysis of Ebay Car Sales Data

### Context
A sample of 50,000 data points were taken from a full dataset of car sale listings on the German eBay website. The data was originally on Kaggle, and had already been cleaned. Dataquest added errors to make it more 'unclean', in order to practice cleaning and data manipulation skills. 


# Questions to Answer
1. Most popular car brands, and corresponding price distributions
2. How strong is the Correlation between registration year and price
3. Distribution of prices vs fuel
4. Effect of damage on price

In [28]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding="Latin-1")

In [29]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

## Data Exploration and Cleaning
### Column Names
We find that the column names can be cleaned to be read easier. We can copy and edit the list, and assign it back to `autos.columns`.

In [30]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [31]:
copy = []
for name in autos.columns:
    copy.append(name.lower())
copy

['datecrawled',
 'name',
 'seller',
 'offertype',
 'price',
 'abtest',
 'vehicletype',
 'yearofregistration',
 'gearbox',
 'powerps',
 'model',
 'odometer',
 'monthofregistration',
 'fueltype',
 'brand',
 'notrepaireddamage',
 'datecreated',
 'nrofpictures',
 'postalcode',
 'lastseen']

In [32]:
autos.columns = ['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price_usd',
 'ab_test',
 'vehicle_type',
 'registration_year',
 'gearbox',
 'power_ps',
 'model',
 'odometer',
 'registration_month',
 'fuel_type',
 'brand',
 'unrepaired_damage',
 'creation_date',
 'num_pictures',
 'postal_code',
 'last_seen']

### Examining Each Column

- `price` and `odometer` columns stored as text
- `seller` and `offer_type` columns have a large number of the same string entries. This gives little analysis value, so is a candidate for being dropped
- `name` is unlikely to yield analysis value so a candidate to be dropped
- `registration_year` contains anomolies which range from '1000' to '9999', and need to be examined
- `power_ps` contains 5500 instances of '0'
- `registration_month` has a minimum of '0'
- `fuel_type` has 4482 missing values
- `unrepaired_damage` has 9829 missing values
- `creation_date` has 1946 instances of the same date and time (likely anomalous unless it's the same seller)
- `num_pictures` column in `.describe` looks odd, so needs further investigation
- `postal_codes` contains 4 unique postal codes which relate to more than 50 listings each

In [33]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,creation_date,num_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-16 21:50:53,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [34]:
autos.head()

,date_crawled,name,seller,offer_type,price_usd,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,creation_date,num_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


#### Converting object columns to numeric datatype

`price_usd` and `odometer` are stored as text due to their additional string characters, which we remove and convert the columns to integer values. The column names are also edited to include measures.

In [35]:
autos["price_usd"] = autos["price_usd"].str.replace("$","")
autos["price_usd"] = autos["price_usd"].str.replace(",","")
autos["price_usd"] = autos["price_usd"].astype(int)

In [36]:
autos["odometer"] = autos["odometer"].str.replace("km","")
autos["odometer"] = autos["odometer"].str.replace(",","")
autos["odometer"] = autos["odometer"].astype(int)

In [37]:
autos["price_usd"].dtype #Confirmed integer

dtype('int32')

In [38]:
autos["odometer"].dtype #Confirmed Integer

dtype('int32')

In [39]:
autos = autos.rename({"odometer":"odometer_km"},axis=1) 
autos.info() #Column names cleaned

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   date_crawled        50000 non-null  object
 1   name                50000 non-null  object
 2   seller              50000 non-null  object
 3   offer_type          50000 non-null  object
 4   price_usd           50000 non-null  int32 
 5   ab_test             50000 non-null  object
 6   vehicle_type        44905 non-null  object
 7   registration_year   50000 non-null  int64 
 8   gearbox             47320 non-null  object
 9   power_ps            50000 non-null  int64 
 10  model               47242 non-null  object
 11  odometer_km         50000 non-null  int32 
 12  registration_month  50000 non-null  int64 
 13  fuel_type           45518 non-null  object
 14  brand               50000 non-null  object
 15  unrepaired_damage   40171 non-null  object
 16  creation_date       50

#### Inspecting `price` in more detail

Upon further inspection, `price` contains 1421 entries that are listed as \\$0. The seller's true expectations aren't captured in this entry, since it may be a startig bid. We will remove these. Note that most of the prices are rounded to the nearest \\$100, \\$500, or \\$1000. This is likely to be the seller's tendency to round. 

A few entries have an asking price of above \\$1 million or more. We check the models of cars above \\$100,000 to verify if they can be classed as luxury or sports cars. 

In [40]:
autos.describe()

,price_usd,registration_year,power_ps,odometer_km,registration_month,num_pictures,postal_code
count,5.000000e+04,50000.000000,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,9.840044e+03,2005.073280,116.355920,125732.700000,5.723360,0.0,50813.627300
std,4.811044e+05,105.712813,209.216627,40042.211706,3.711984,0.0,25779.747957
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1.100000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30451.000000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49577.000000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71540.000000
max,1.000000e+08,9999.000000,17700.000000,150000.000000,12.000000,0.0,99998.000000


In [41]:
print(autos["price_usd"].isnull().value_counts()) #No missing data in Price column
autos["price_usd"].value_counts().head(10)    #1421 data points priced at $0, which makes up approx. 2.8% of data

False    50000
Name: price_usd, dtype: int64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
Name: price_usd, dtype: int64

In [42]:
1421/50000

0.02842

In [43]:
autos["price_usd"].value_counts().sort_index(ascending=False).head(10)
   #many listings with unrealistic pricing, likely anomalies and candidates for dropping

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
Name: price_usd, dtype: int64

#### Analysis and removal of Anomolous Prices
We want to examine some of the rows which appear to be anomolous in price before dropping them. We would like to minimise data loss as much as possible, but balance this with how much time it will take to process the data. 

We examine the listings priced above $100,000 to see whether they are reasonable prices. Most seem to be Porsche 911 which is a sports model which sometimes comes in rare or limited edition models. Hence we will leave those rows. 

Many rows have `brand` as 'sontige_autos' or `model` as 'andere' which translates to other vehicles. Since Their prices don't look too unreasonable, these may also be accurately priced. 

The Mercedes Benz G-Class priced at $169999 seems high, but [some luxury models](https://www.mbusa.com/en/vehicles/class/g-class/suv/type-amg) are being sold around that price point. Similarly, several BMW Z-class listings are priced high, but there are several classic models that have a high market value, as can be seen from the image below from [classic-trader.com](https://www.classic-trader.com/uk/cars/search/bmw/z8).
![alt text](zclass.png "Title")

The first anomolous result is the Volkswagen Jetta, and the latter cars seem unreasonably priced also, since they are relatively cheap and common car models. We drop the rows priced from $999,990 upward by selecting the remaining rows and assigning it back to `autos`.

In [44]:
few_columns = ["brand","model","vehicle_type","price_usd","odometer_km","registration_year"]
(
    autos.loc[autos["price_usd"]>=100000 , few_columns]
                .sort_values("price_usd",axis=0)
)

,brand,model,vehicle_type,price_usd,odometer_km,registration_year
29286,porsche,911,coupe,104900,30000,2011
17540,porsche,911,coupe,105000,150000,1988
16964,sonstige_autos,NaN,coupe,105000,80000,2010
49391,sonstige_autos,NaN,coupe,109999,30000,2008
22060,sonstige_autos,NaN,suv,114400,5000,2016
7402,porsche,911,cabrio,115000,5000,2016
21783,porsche,911,cabrio,115991,10000,2013
33884,porsche,911,cabrio,116000,30000,2013
38814,porsche,911,coupe,119500,150000,1992
43282,porsche,911,cabrio,119900,30000,2014


In [45]:
autos = autos[autos["price_usd"].between(1,999990)] #Dropping rows where price is 0 or greater than 999990
autos["price_usd"].describe()

count     48566.000000
mean       5909.404666
std       10120.652480
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      999990.000000
Name: price_usd, dtype: float64

#### Inspecting Date Columns
We notice that `registration_year` and `registration_month` are stored as integers, whereas `last_seen`,`creation_date` and `date_crawled` are all stored as string objects. 

Converting `last_seen`,`creation_date` and `date_crawled` to numeric can allow us to perform analyses. First we understand how they're formatted. The first 10 characters of these 3 columns describe the date, and the last 8 describe the time. We can seperate these to perform analysis by converting this to a numeric datatype. 


In [46]:
print(autos["registration_year"].dtype)
print(autos["registration_month"].dtype)
print(autos["date_crawled"].dtype)
print(autos["creation_date"].dtype)
print(autos["last_seen"].dtype)

int64
int64
object
object
object


In [47]:
autos[["date_crawled","creation_date","last_seen","registration_year","registration_month"]].head()

,date_crawled,creation_date,last_seen,registration_year,registration_month
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54,2004,3
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08,1997,6
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37,2009,7
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28,2007,6
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50,2003,7


In [48]:
autos["date_crawled"].str[0:10].head()

0    2016-03-26
1    2016-04-04
2    2016-03-26
3    2016-03-12
4    2016-04-01
Name: date_crawled, dtype: object

#### Analysing and Cleaning the Registration Year Column

We find that many entries for registration year are clearly anomalous looking at minimum and maximum. It's impossible to have a registration year after the data had been crawled, so we can remove registration years greater than 2006. We find 5 listings with registration year before 1900 which is highly unlikely so we begin by removing those. 

In [20]:
autos["registration_year"].describe()

count    48566.000000
mean      2004.755014
std         88.643020
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [21]:
autos["registration_year"].unique()

array([2004, 1997, 2009, 2007, 2003, 2006, 1995, 1998, 2000, 2017, 2010,
       1999, 1982, 1990, 2015, 2014, 1996, 1992, 2002, 2012, 2011, 2005,
       2008, 1985, 2016, 1994, 1986, 2001, 2018, 2013, 1972, 1993, 1988,
       1989, 1973, 1967, 1976, 4500, 1987, 1991, 1983, 1960, 1969, 1950,
       1978, 1980, 1984, 1963, 1977, 1961, 1968, 1934, 1965, 1971, 1966,
       1979, 1981, 1970, 1974, 1910, 1975, 5000, 4100, 2019, 1956, 9999,
       6200, 1964, 1959, 1958, 1800, 1948, 1931, 1943, 1941, 1962, 1927,
       1937, 1929, 1000, 1957, 1952, 1111, 1955, 1939, 8888, 1954, 1938,
       2800, 5911, 1953, 1951, 4800, 1001, 9000], dtype=int64)

In [22]:

autos["registration_year"].value_counts().sort_index().head(20) 

1000    1
1001    1
1111    1
1800    2
1910    5
1927    1
1929    1
1931    1
1934    2
1937    4
1938    1
1939    1
1941    2
1943    1
1948    1
1950    3
1951    2
1952    1
1953    1
1954    2
Name: registration_year, dtype: int64

In [23]:
autos = autos.loc[autos["registration_year"].between(1900,2006)]

In [24]:
cols = ["registration_year","brand","model","price_usd"]

autos.loc[autos["registration_year"]<1950,cols].sort_values("registration_year")


,registration_year,brand,model,price_usd
45157,1910,trabant,NaN,15
30781,1910,opel,calibra,30
3679,1910,sonstige_autos,NaN,1
28693,1910,renault,NaN,599
22659,1910,opel,corsa,500
21416,1927,ford,andere,16500
22101,1929,bmw,andere,11500
11246,1931,ford,andere,27500
2573,1934,sonstige_autos,NaN,3000
2221,1934,opel,andere,3350


#### Inspecting `registration_year`
By using wikipedia we can inspect some of the listings to see if that model was produced during the registration year or not.

Trabant didn't produce cars until the 1950s.
The Opel Calibra started being produced in 1989.

The first realistically possible listing was the 1927 Ford priced at \\$16500, so we will remove all listings with registration date smaller than 1927.

In [25]:
autos = autos.loc[autos["registration_year"].between(1927,2006)]

In [26]:
autos["registration_year"].describe() 

count    32500.000000
mean      1999.670892
std          5.894761
min       1927.000000
25%       1998.000000
50%       2000.000000
75%       2004.000000
max       2006.000000
Name: registration_year, dtype: float64

#### Analysing Registration Month 
We find the minimum is 0, so that is missing values. 4480, nearly 10% of our dataset has `registration_month` of 0, which is probably caused by missing values. However, these don't necessarily have to be removed for our analysis. 

We might want to think about converting the 0 values to null values to perform more accurate numerical analysis.


In [49]:
autos["registration_month"].describe() 

count    48566.000000
mean         5.782379
std          3.685665
min          0.000000
25%          3.000000
50%          6.000000
75%          9.000000
max         12.000000
Name: registration_month, dtype: float64

In [50]:
autos["registration_month"].value_counts() 

3     5003
0     4480
6     4271
4     4036
5     4031
7     3857
10    3588
12    3375
9     3330
11    3313
1     3219
8     3126
2     2937
Name: registration_month, dtype: int64

#### Dropping `seller`, `offer_type` and `num_pictures`

As we see below, the `seller`, `offer_type` and `num_pictures` are text columns which contain very little analytic value, so these can be dropped


In [34]:
autos["seller"].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [ ]:
autos["offer_type"].value_counts()

In [61]:
autos["num_pictures"].value_counts()

0    50000
Name: num_pictures, dtype: int64

In [59]:
autos = autos.drop(columns = ["seller","offer_type","num_pictures"])

####

For `odometer_km`, there are only 13 individual values which suggests that they are ranges selected from a list instead of precise values. This will decrease the strength of our conclusions based on this column.

In [25]:
autos["odometer_km"].unique()

array([150000,  70000,  50000,  80000,  10000,  30000, 125000,  90000,
        20000,  60000,   5000, 100000,  40000])

In [57]:
autos["unrepaired_damage"].isnull().value_counts()

False    40171
True      9829
Name: unrepaired_damage, dtype: int64

In [36]:
autos["ab_test"].value_counts()

test       25756
control    24244
Name: ab_test, dtype: int64

In [50]:
autos["power_ps"].value_counts()

0        5500
75       3171
60       2195
150      2046
140      1884
         ... 
650         1
490         1
362         1
153         1
16312       1
Name: power_ps, Length: 448, dtype: int64

In [54]:
print(autos["fuel_type"].unique()) #null values
autos["fuel_type"].isnull().value_counts() #4482 missing values

['lpg' 'benzin' 'diesel' nan 'cng' 'hybrid' 'elektro' 'andere']


False    45518
True      4482
Name: fuel_type, dtype: int64

In [65]:
autos["postal_code"].value_counts()

10115    109
65428    104
66333     54
45888     50
44145     48
        ... 
23942      1
83365      1
95683      1
97794      1
67585      1
Name: postal_code, Length: 7014, dtype: int64

# Analysis of Data

## To include:
#### Distribution of Registration years
#### Distribution of Prices
#### Histogram of German brands and International vs price

First we look at `date_crawled` by finding the relative proportion of each date including null values. Then we sort by date. We repeat the process for the other 2 columns also. 

Most of the listings were crawled on 3rd of April 2016, making up 3.9% of the total dataset. However, the proportions seem fairly uniform, suggesting the crawl took place at a similar pace from 5th March to 7th April 2016.

In [80]:
(autos["date_crawled"]
         .str[0:10]
         .value_counts(normalize=True, dropna=False)
         .sort_index()
         )

2016-03-05    0.025387
2016-03-06    0.013944
2016-03-07    0.035970
2016-03-08    0.033269
2016-03-09    0.033209
2016-03-10    0.032129
2016-03-11    0.032489
2016-03-12    0.036770
2016-03-13    0.015564
2016-03-14    0.036630
2016-03-15    0.033990
2016-03-16    0.029508
2016-03-17    0.031509
2016-03-18    0.013064
2016-03-19    0.034910
2016-03-20    0.037831
2016-03-21    0.037490
2016-03-22    0.032909
2016-03-23    0.032389
2016-03-24    0.029108
2016-03-25    0.031749
2016-03-26    0.032489
2016-03-27    0.031049
2016-03-28    0.034850
2016-03-29    0.034150
2016-03-30    0.033629
2016-03-31    0.031909
2016-04-01    0.033809
2016-04-02    0.035410
2016-04-03    0.038691
2016-04-04    0.036490
2016-04-05    0.013104
2016-04-06    0.003181
2016-04-07    0.001420
Name: date_crawled, dtype: float64

`creation_date` column shows that a small proportion of listings were started as far back as June 2015. However the vast majority were listed in 2016. It's useful to note, though, that sellers could have taken off the listing and put the listing back up, especially in auctions where no bids were received. 

In [83]:
(autos["creation_date"]
         .str[0:10]
         .value_counts(normalize=True,dropna=False)
         .sort_index()
         )

2015-06-11    0.000020
2015-08-10    0.000020
2015-09-09    0.000020
2015-11-10    0.000020
2015-12-05    0.000020
                ...   
2016-04-03    0.038931
2016-04-04    0.036850
2016-04-05    0.011843
2016-04-06    0.003261
2016-04-07    0.001280
Name: creation_date, Length: 76, dtype: float64

The `last_seen` column tells us the last date at which any particular listing was visible on the site. This can be interpreted as when the car was likely sold, so the listing was removed from the website. The distribution seems fairly uniform until we reach the last three days of crawling. These number likely coincide with the crawling period ending as opposed to a sudden huge boost to sales during those days. 

In [74]:
(autos["last_seen"]
         .str[0:10]
         .value_counts(normalize=True,dropna=False)
         .sort_index()
         )

2016-03-05    0.001080
2016-03-06    0.004421
2016-03-07    0.005362
2016-03-08    0.007582
2016-03-09    0.009843
2016-03-10    0.010763
2016-03-11    0.012524
2016-03-12    0.023807
2016-03-13    0.008983
2016-03-14    0.012804
2016-03-15    0.015884
2016-03-16    0.016445
2016-03-17    0.027928
2016-03-18    0.007422
2016-03-19    0.015744
2016-03-20    0.020706
2016-03-21    0.020726
2016-03-22    0.021586
2016-03-23    0.018585
2016-03-24    0.019565
2016-03-25    0.019205
2016-03-26    0.016965
2016-03-27    0.016024
2016-03-28    0.020846
2016-03-29    0.022326
2016-03-30    0.024847
2016-03-31    0.023827
2016-04-01    0.023106
2016-04-02    0.024887
2016-04-03    0.025367
2016-04-04    0.024627
2016-04-05    0.124275
2016-04-06    0.220982
2016-04-07    0.130957
Name: last_seen, dtype: float64